In [44]:

lyrics = r"""
Lyrics: *(Whistles)*
Once a sloth named Jett    \
In trees he swung, but aimed skyward he'd set
Trained so hard, *through skies he flew*
To the stars, his dreams came true

He soared past planets, beyond the moon  \
Space sloth Jett, in a cosmic tune  \
Fighting threats across the stars  \
Saving galaxies near and far

A hero’s cheer in the Lily Way night  \
Jett saved the day, with all his might  \
Up in the cosmos, where heroes play  \
Our sloth Jett leads the way

*(Cosmic booms, cheers in the galaxy)*
"""


In [55]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, AudioFileClip
from moviepy.video.fx.loop import loop
import ffmpeg
import asyncio
import time
import os

async def merge_videos_and_song_with_subtitles(srt_path):
    # Step 1: Download song and videos
    # async with ClientSession() as session:
    #     song_path = await download_file(session, song_url, 'song.mp3')
    #     video_paths = []
    #     for i, video_url in enumerate(video_urls):
    #         video_path = await download_file(session, video_url, f'video_{i+1}.mp4')
    #         video_paths.append(video_path)

    song_path = './song.mp3'
    video_paths = [f'./video_{i+1}.mp4' for i in range(2)]

    # Step 2: Load videos using MoviePy
    video_clips = [VideoFileClip(video) for video in video_paths]
    
    # Step 3: Concatenate and loop the videos until they match the song duration
    concatenated_clip = concatenate_videoclips(video_clips)

    song_audio = AudioFileClip(song_path)
    song_duration = song_audio.duration
    final_video_clip = loop(concatenated_clip, duration=song_duration)
    final_video_clip = final_video_clip.set_audio(song_audio)

    # Step 4: Add subtitles using ffmpeg (MoviePy)
    # MoviePy uses ffmpeg to overlay the subtitles without ImageMagick.
    final_video_with_subtitles = final_video_clip.subclip(0, song_duration).set_duration(song_duration).set_audio(song_audio)

    # Step 5: Write the output to a file
    output_path = f"final_output_with_subtitles_{time.time()}.mp4"
    final_video_with_subtitles.write_videofile(output_path, codec='libx265', audio_codec='aac', ffmpeg_params=['-vf', f"subtitles={srt_path}", '-preset', 'ultrafast'])

    # Step 6: Clean up temporary files
    song_audio.close()
    final_video_clip.close()
    concatenated_clip.close()
    final_video_with_subtitles.close()
    for video in video_clips:
        video.close()
    
    return output_path

In [56]:
await merge_videos_and_song_with_subtitles('./output.ass')


Moviepy - Building video final_output_with_subtitles_1729436848.9809334.mp4.
MoviePy - Writing audio in final_output_with_subtitles_1729436848.9809334TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video final_output_with_subtitles_1729436848.9809334.mp4



Moviepy - Done !
Moviepy - video ready final_output_with_subtitles_1729436848.9809334.mp4


'final_output_with_subtitles_1729436848.9809334.mp4'

In [16]:
import stable_whisper

In [48]:
model = stable_whisper.load_model("base")

In [49]:
# remove everything in brackets of any sory from lyrics
import re
lyrics = re.sub(r'\[.*?\]|\(.*?\)', '', lyrics)


In [70]:
result = model.align('./song.mp3', lyrics, language='en', fast_mode=True, regroup='cm_sp=,* /，/\n/\\_sg=.5_sp=.* /。/?/？')


Align: 100%|██████████| 174.02/174.02 [00:00<00:00, 238.34sec/s]
Adjustment: 100%|██████████| 89.52/89.52 [00:00<00:00, 44816.67sec/s]


In [71]:
for segment in result:
    print("[%.2f -> %.2f] %s" % (segment.start, segment.end, segment.text))



[9.62 -> 13.08]  Lyrics: ** Once a sloth named Jett
[13.94 -> 13.96]     \
[13.96 -> 15.62]  In trees he swung,
[15.82 -> 20.64]  but aimed skyward he'd set Trained so hard,
[20.90 -> 25.70]  *through skies he flew* To the stars,
[26.20 -> 28.06]  his dreams came true
[29.78 -> 30.60]   He soared past planets,
[31.58 -> 34.76]  beyond the moon  \
[34.76 -> 36.38]  Space sloth Jett,
[36.38 -> 38.94]  in a cosmic tune  \
[38.94 -> 44.24]  Fighting threats across the stars  \
[44.24 -> 53.26]  Saving galaxies near and far  A hero’s cheer in the Lily Way night  \
[53.26 -> 54.18]  Jett saved the day,
[54.32 -> 55.90]  with all his might  \
[55.90 -> 56.84]  Up in the cosmos,
[56.84 -> 58.40]  where heroes play  \
[58.40 -> 64.62]  Our sloth Jett leads the way  ** 


In [54]:
srt = result.to_ass('output.ass', karaoke=True)


Saved: c:\Users\pmazu\Documents\GitHub\BlinkStory\output.ass
